In [32]:
// hacky way to set java path since macos breaks this
val jniPath = "/Users/shawnmeier/software/z3/build"
val newPath = Array(jniPath) ++  System.getProperty("java.library.path").split(":")
System.setProperty("java.library.path",newPath.distinct.mkString(":"))
//set sys_paths to null so that java.library.path will be reevaluated next time it is needed
val sysPathsField = classOf[ClassLoader].getDeclaredField("sys_paths");
sysPathsField.setAccessible(true);
sysPathsField.set(null, null);

val path = "/Users/shawnmeier/Documents/source/bounder/soot_hopper/target/scala-2.13/soot_hopper-assembly-0.1.jar"
//unzip -p jdiff.jar META-INF/MANIFEST.MF
interp.load.cp(os.Path(path))

import $ivy.`org.plotly-scala:plotly-almond_2.13:0.8.2`
import $ivy.`com.github.pathikrit::better-files:3.9.1`

import $ivy.`com.lihaoyi:ujson_2.13:1.3.8`
// import $ivy.`edu.colorado.plv.bounder:soot_hopper_2.13:0.1`

import plotly._, element._, layout._, Plotly._
import ujson.Value
import sys.process._


jniPath: String = "/Users/shawnmeier/software/z3/build"
newPath: Array[String] = Array(
  "/Users/shawnmeier/software/z3/build",
  "/Users/shawnmeier/software/z3/build",
  "/Users/shawnmeier/Library/Java/Extensions",
  "/Library/Java/Extensions",
  "/Network/Library/Java/Extensions",
  "/System/Library/Java/Extensions",
  "/usr/lib/java",
  "."
)
res31_2: String = "/Users/shawnmeier/software/z3/build:/Users/shawnmeier/Library/Java/Extensions:/Library/Java/Extensions:/Network/Library/Java/Extensions:/System/Library/Java/Extensions:/usr/lib/java:."
sysPathsField: java.lang.reflect.Field = private static java.lang.String[] java.lang.ClassLoader.sys_paths
path: String = "/Users/shawnmeier/Documents/source/bounder/soot_hopper/target/scala-2.13/soot_hopper-assembly-0.1.jar"
import $ivy.$                                          

import $ivy.$                                         


import $ivy.$                             
// import $ivy.`edu.colorado.plv.bounder:soot_hopper_2.13:0.1`



In [33]:
import better.files._

import scala.util.Random
import edu.colorado.plv.bounder.Driver
import edu.colorado.plv.bounder.Action
import edu.colorado.plv.bounder.RunConfig
import upickle.default.read
import upickle.default.write
import scala.collection.parallel.CollectionConverters.{ImmutableSetIsParallelizable, IterableIsParallelizable}
import edu.colorado.plv.bounder.ExperimentsDb
import edu.colorado.plv.bounder.BounderUtil
import scala.concurrent.duration._
import scala.language.postfixOps
import slick.driver.H2Driver.api._
import slick.jdbc.GetResult
import slick.jdbc.SQLActionBuilder
import scala.concurrent.Await
Driver.setZ3Path("/Users/shawnmeier/software/z3/build")
BounderUtil.setEnv(Map("DYLD_LIBRARY_PATH" -> "/Users/shawnmeier/software/z3/build","ANDROID_HOME" -> "/Users/shawnmeier/Library/Android/sdk","HOME" -> "/Users/shawnmeier"))

java.library.path set to: /Users/shawnmeier/software/z3/build:/Users/shawnmeier/Library/Java/Extensions:/Library/Java/Extensions:/Network/Library/Java/Extensions:/System/Library/Java/Extensions:/usr/lib/java:.


import better.files._


import scala.util.Random

import edu.colorado.plv.bounder.Driver

import edu.colorado.plv.bounder.Action

import edu.colorado.plv.bounder.RunConfig

import upickle.default.read

import upickle.default.write

import scala.collection.parallel.CollectionConverters.{ImmutableSetIsParallelizable, IterableIsParallelizable}

import edu.colorado.plv.bounder.ExperimentsDb

import edu.colorado.plv.bounder.BounderUtil

import scala.concurrent.duration._

import scala.language.postfixOps

import slick.driver.H2Driver.api._

import slick.jdbc.GetResult

import slick.jdbc.SQLActionBuilder

import scala.concurrent.Await


In [34]:
val expDir = File("/Users/shawnmeier/Documents/source/bounder/experiments/5fdroid5dev_sampderef")
val baseDir = File("/Users/shawnmeier/Documents/data/fdroid")

expDir: File = /Users/shawnmeier/Documents/source/bounder/experiments/5fdroid5dev_sampderef
baseDir: File = /Users/shawnmeier/Documents/data/fdroid

Create Config Files From List of APKs
-------------------------------------

In [35]:
// Create json files for config runs
System.setProperty("user.dir", "/Users/shawnmeier/Documents/source/bounder/soot_hopper");
def createConfigsFromList():Unit = {
    val mode = Driver.SampleDeref
    val baseConfig = RunConfig(limit = -1, samples = 50)
    case class ApkSample(apk:String, pkg:String, outDir:File)
    val apkSamples = (expDir / "appSample.txt")
        .contentAsString
        .split("\n")map{v => 
            val pkg = v.split(":")(1)
            println(s"package: $pkg")
            val apk = v.split(":")(0)
            val apkDirCfg = apk.toString.replace(baseDir.toString,"${baseDir}")
            val outDirCfg = "${baseDirOut}/" + pkg
            val outDir = expDir / pkg
//             val getSample = outDir / "getSample.json"
//             if(getSample.exists())getSample.delete()
            // val samples = if(pkg.contains("lyonbros")) 90 else 200
            val cfg = baseConfig.copy(apkPath=apkDirCfg, outFolder=Some(outDirCfg), samples=90) // 5 dereference samples per class
//             getSample.append(write[RunConfig](cfg))
            outDir.createIfNotExists(true)
            // sample dereference locations
            Action(Driver.SampleDeref, Some(expDir.toString), Some(baseDir.toString),
                   cfg, filter=Some(pkg))
        }
    apkSamples.foreach{action => 
        println(s"action: ${action}")
        Driver.runAction(action)
    }
}
//createConfigsFromList()

res34_0: String = "/Users/shawnmeier/Documents/source/bounder/soot_hopper"
defined function createConfigsFromList

In [36]:
expDir.glob("**/*.json").toList.size


res35: Int = 90

Upload needed files to database
-------------------------------

In [37]:
val db = new ExperimentsDb(Some("/Users/shawnmeier/Documents/source/bounder/soot_hopper/target/scala-2.13/soot_hopper-assembly-0.1.jar"))

Initializing database


db: ExperimentsDb = edu.colorado.plv.bounder.ExperimentsDb@3c6deeea

In [38]:
// Upload data of APK files to database for experiments
def uploadApks() = {
    def appList = (expDir / "appSample.txt").contentAsString.split("\n")
        .map{v => 
            val spl = v.split(":")
            val name = spl(0).replace(baseDir.toString,"")
            val path = spl(0)
            (name,path)
        }
    appList.foreach(a => db.uploadApk(a._1,File(a._2)))
}
uploadApks()

defined function uploadApks

Empty spec set all deref
========================

In [39]:
// Create empty spec run config
val emptyCfg = db.createConfig(
    File("/Users/shawnmeier/Documents/source/bounder/soot_hopper/specs/emptySpec.txt"),
    File("/Users/shawnmeier/Documents/source/bounder/soot_hopper/target/scala-2.13/soot_hopper-assembly-0.1.jar"),
    "Empty spec set"
)

emptyCfg: Int = 16

In [40]:
// Utilities for manipulating run configs

def splitConfig(cfg:File, maxJobsPerFile:Int) = {
    val cfgContent = read[RunConfig](cfg.contentAsString)
    val parentDir = cfg.parent
    val cfgName = cfg.name.split('.')(0)
    assert(cfg.name.split('.')(1) == "json", "Input should have ext json")
    val newCfg = cfgContent.initialQuery.grouped(maxJobsPerFile).zipWithIndex
    
    newCfg.foreach{
        case (iqs,ind) => 
            val outfname = s"${cfgName}_${ind}.json"
            println(s"index: ${ind} size: ${cfg.size} fname: ${outfname}")
            val outf = parentDir / outfname
            outf.overwrite(write[RunConfig](cfgContent.copy(initialQuery=iqs)))
    }
}

// update timeout in run config
def setTimeout(nt:Int) = {
    val configs = expDir.glob("**/*.json")
    configs.foreach{cfgf =>
        val cfg = read[RunConfig](cfgf.contentAsString)
        cfgf.delete()
        cfgf.overwrite(write[RunConfig](cfg.copy(timeLimit=nt)))
    }
//     write[RunConfig](.map{c => read[RunConfig](c.contentAsString)}.toList.head.copy(timeLimit = 180))
}
setTimeout(180)
//splitConfig(File("/Users/shawnmeier/Documents/source/bounder/experiments/5fdroid5dev_sampderef/com.lyonbros/sample.json"),5)

defined function splitConfig
defined function setTimeout

In [41]:
// push experiment .json files to database
def pushExp() = {
    val experiments = expDir.glob("**/*.json").toList
    println(experiments.size)
    experiments.foreach(v => db.createJob(v, Some("empty1"), emptyCfg))
}
pushExp()

90


defined function pushExp

Bulk Job Tasks
==============

In [ ]:
case class Count(n:Int)
case class StrRes(s:String)
implicit val getCountResult = GetResult(r => Count(r.<<))
implicit val getStrResult = GetResult(r => StrRes(r.<<))

def pauseJobs() = {
    val pauseJobsQ = sql"""update jobs set status='paused' where status='new';"""
    Await.result(db.db.run(pauseJobsQ.as[StrRes]), 30 seconds)
}
def resumeJobs() = {
    val pauseJobsQ = sql"""update jobs set status='new' where status='paused';"""
    Await.result(db.db.run(pauseJobsQ.as[StrRes]), 30 seconds)
}
def clearInputs() = {
    val clearInputsQ = sql"""delete from apks where apkname like 'jar_%' or apkname like 'spec_%';"""
    Await.result(db.db.run(clearInputsQ.as[StrRes]), 30 seconds)
    val clearInputsTableQ = sql"""delete from inputs;"""
    Await.result(db.db.run(clearInputsTableQ.as[StrRes]), 30 seconds)
}
def resetJobs() = {
    val resetJobsQ = sql"""update jobs set inputid=4, stderr='',stdout='',owner='',started=NULL,ended=NULL,status='new';"""
    Await.result(db.db.run(resetJobsQ.as[StrRes]), 30 seconds)
}
def clearTable(tableName:String) = {
    val clear = sql"""delete from #$tableName ;"""
    Await.result(db.db.run(clear.as[StrRes]), 30 seconds)
}
def clearApks() = {
    
}
def clearResults(){
    val clearResults = sql"""delete from results; delete from resultdata;"""
    Await.result(db.db.run(clearResults.as[StrRes]), 30 seconds)
}
def clearAll(){
    clearInputs()
    clearResults()
    clearTable("jobs")
    clearTable("apks")
}

//clearResults()
pauseJobs()
//clearAll()
//resumeJobs()
//Await.result(db.db.run(sql"""select count(*) from results;""".as[Count]), 30 seconds)

In [ ]:
//db.downloadApk("/com.gpl.rpg.AndorsTrail/0.7.5/apk/com.gpl.rpg.AndorsTrail_46.apk", File("/Users/shawnmeier/Desktop/testDownload.apk"))